<a href="https://colab.research.google.com/github/aditi1209/Anime-Recommendation/blob/master/Anime_Recommendation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
pip install meaningcloud-python

In [0]:
import pandas as pd
import tweepy as tw
import meaningcloud as mc

In [0]:
#@title
consumer_key = 'Enter twitter consumer key'
consumer_secret_key = 'Enter twitter consumer secret key'
access_token = 'Enter access token'
access_token_secret = 'Enter access secret token'

In [0]:
#variable intialization
tweet_keywords = set()
all_tweets = ''
license_key = 'Enter meaning cloud license key'
model = 'IAB_en'

In [0]:
auth = tw.OAuthHandler(consumer_key= consumer_key, consumer_secret= consumer_secret_key)

In [0]:
auth.set_access_token(access_token, access_token_secret)

In [0]:
api = tw.API(auth, wait_on_rate_limit=True)

In [0]:
username = input("Enter the twitter handle: ")

In [0]:
#finding out the tweets of a person whose handle is provided
tweets = tw.Cursor(api.user_timeline, id = username, lang = 'en').items()
print(tweets)

In [0]:
#merging all tweets together in a single string
for tweet in tweets:
  all_tweets = all_tweets + ' ' + tweet.text
all_tweets

In [0]:
#Extracting concepts from the string
keywords = mc.TopicsResponse(mc.TopicsRequest(license_key, txt= all_tweets, topicType='c', lang= 'en').sendReq())
concepts = keywords.getConcepts()
print(concepts)

In [0]:
#Concepts are put into a set
for concept in concepts:
  tweet_keywords.add(keywords.getTopicForm(concept))
print(tweet_keywords)

In [0]:
total_keywords = len(tweet_keywords)
print(total_keywords)

In [0]:
#@title
all_anime = pd.read_csv('animes.csv')
all_anime['keyword_count'] = int(0)
all_anime['final_weighted_value'] = int(0)


In [0]:
for anime in all_anime['synopsis']:
  count = 0
  if anime != '':
    for tweet_keyword in tweet_keywords:
      if tweet_keyword in str(anime):
        count = count + 1
  all_anime['keyword_count'].loc[all_anime['synopsis'] == anime] = count
all_anime



In [0]:
#for count in all_anime['keyword_count']:
#  if count == '':
#    all_anime.loc[all_anime['keyword_count']=='','keyword_count'] = int(0)
all_anime['final_weighted_value'] = all_anime['keyword_count']*0.2+all_anime['score']*0.8
max_weighted_value = all_anime['final_weighted_value'].max()
all_anime[all_anime['final_weighted_value'] == max_weighted_value]

In [0]:
all_anime.sort_values(by= 'final_weighted_value', ascending= False).head(10)